## Bot from Scratch using Telegram API

In [1]:
import requests
import numpy as np
import json
import time

In [2]:
API_token = '1704677561:AAEDL-fs6ekIoXrgj_lHhab3V1zq2hEcJXE'

In [158]:
class telegram_bot():
    def __init__(self, token):
        self.token = token
        self.base = f'https://api.telegram.org/bot{self.token}/'
    
    def get_updates(self, offset=None):
        url = self.base+'getUpdates?timeout=100'
        if offset:
            url = url + f'&offset={offset+1}'
        r = requests.get(url)
        return r.json()
    
    def send_message(self, message, chat_id):
        url = self.base+f'sendMessage?chat_id={chat_id}&text={message}'
        if message is not None:
            r = requests.get(url)
        return r.json()

### Reply with upper case of the string received as an update to the BOT

Initialising the update id to `None`.
- We will wait until we get a new update inside a `while True` loop.
- if we do not get an update, we wont get inside the `if` condition and will not execute the `for` loop.
- As soon as we get an update, we will get inside the `if` condition and check the updates for any messages.
- Once we get the message as a text, we send the reply as an upper case of the same text to the sender.
- All this happens over a timeout of 100s. If the sender doesnot send anything for 100s the update command shall stop receiving.

In [1]:
update_id=None
while True:
    updates = bot.get_updates(update_id)
    if updates:
        updates_res = updates['result']
        for item in updates_res:
            update_id = item['update_id']
            try:
                mssg_body = item['message']
                mssg=mssg_body['text']
                sender=mssg_body['from']['id']
            except:
                mssg=None
            if mssg is not None:
                reply = mssg.upper()
            bot.send_message(reply, sender)

NameError: name 'bot' is not defined

## Bot using Telegram Python Bot library

In [1]:
import telegram
from telegram import Bot
import datetime
from telegram.ext import Updater, Dispatcher, CommandHandler, MessageHandler, Filters
import requests
import numpy as np
import time
from selenium import webdriver

In [2]:
from scrappy_MMT import GET_DATA, CLEAN_DATA
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [3]:
API_token = '1704677561:AAEDL-fs6ekIoXrgj_lHhab3V1zq2hEcJXE'
bot = Bot(token=API_token)

Print basic information about the bot using `get_me` method.

In [4]:
print(bot.get_me())

{'id': 1704677561, 'first_name': 'sampleBOT', 'is_bot': True, 'username': 'panda_23_panda_bot', 'can_join_groups': True, 'can_read_all_group_messages': False, 'supports_inline_queries': False}


In [5]:
updater = Updater(token=API_token, use_context=True)
dispatcher = updater.dispatcher

`Updater` class receives updates from the bot, and the inbuilt `Dispatcher` object it uses handles the types of updates that come from the bot.

The following function is the function executed when `/start` command is given to the bot.

`dispatcher` instance handles the command by executing the function for that particular command.

In [6]:
def start(update, context):
    context.bot.send_message(chat_id=update.effective_chat.id, text='Hello, nice to meet you. I am a work under progress...')

start_handler = CommandHandler('start', start)
dispatcher.add_handler(start_handler)

In [7]:
def help_command(update, context):
    pic = open('pic.jpg', 'rb')
    response = bot.sendPhoto(chat_id=update.effective_chat.id, photo=pic, caption='Hey There!')

help_handler = CommandHandler('help', help_command)
dispatcher.add_handler(help_handler)

In [8]:
def echo(update, context):
    context.bot.send_message(chat_id=update.effective_chat.id, text=update.message.text.upper())

echo_handler = MessageHandler(Filters.text & (~Filters.command), echo)
dispatcher.add_handler(echo_handler)

In [9]:
def getPrice_command(update, context):
    # Get Prices
    today = datetime.datetime.today().strftime('%d/%m/%Y')
    cities = ['BLR', 'DEL']
    tag = datetime.datetime.today().strftime('%d/%m/%Y_%H:%M:%S')
    slider_text = GET_DATA(today, cities)
    info_df = CLEAN_DATA(slider_text, tag)
    
    # Create Plotly graph
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=info_df['Date'], y=info_df[f'Price_({tag})'].astype('int32'),
                             name = 'Minmium price of flights'))
    fig.update_layout(title='Min Flight Prices (including all airlines) as listed on MMT site', xaxis_title = 'Date', yaxis_title = 'Price (in Ruppees)')
    fig.write_html("Prices_trend.html")
    
    # Send Plotly graph as html file 
    html_file = open('Prices_trend.html', 'rb')
    response = bot.sendDocument(chat_id=update.effective_chat.id, document=html_file, caption='Prices trend from today...')
    
getPrice_handler = CommandHandler('getprices', getPrice_command)
dispatcher.add_handler(getPrice_handler)

In [ ]:
def getMinPrices(update, context):
    

In [10]:
updater.start_polling()